# DROIC Calculator
<img src="DROIC_signal_chain_crop.png" width=600>

In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
from decimal import Decimal
import matplotlib.pyplot as plt
%matplotlib inline

## Constants and parameters

# Constants
q = 1.602e-19 # [C], Elementary charge
kB = 1.38e-23 # [J/K], Boltzmann's constant

# Calculation parameters
t = np.logspace(-9,-1,1000) # [s], Integration time vector
t1 = np.ones(t.shape) # Ones vector of size(t)


## "Set Parameters" section heading
SetParamHdg = widgets.HTML(value = "<p style='font-size:16pt'><b>Set Parameters</b></p>")
display(SetParamHdg)


## Plot function
def plotfun(Isig,Idk,Vn_Amp_Ampin,Rdata,Npix,T,CifF,Vmax,Res_ADC):

    ## Parse parameters
    Ci = CifF * 1e-15 # [F], Integration capacitance (get value in F)
    Vn_Amp_Ampin = Vn_Amp_Ampin * t1 # [Vrms], Amplifier noise referred to amplifier input (multiply by ones vector)

    
    ## Compute noise and SNR

    # Total pixel current
    Itot = Isig + Idk # [A]

    # ADC conversion gain
    G_ADC = 2**Res_ADC / Vmax # [DU/V], ADC gain

    # Quantization noise
    Dn_quant_out = 1/np.sqrt(12) * t1 # [DUrms]

    # Integrated pixel voltage
    Vtot_pix = np.minimum(Itot * t /Ci,Vmax) # [V], Total
    Vsig_pix = Vtot_pix * Isig/Itot # [V], Signal only

    # Pixel noise sources
    Vn_shot_pix = np.sqrt(q * Vtot_pix /Ci) * t1 # [Vrms], Shot noise
    Vn_kTC_pix = np.sqrt(kB * T /Ci) * t1 # [Vrms], kTC (reset) noise

    # Propagate signal to output
    Dsig_out = Vsig_pix * G_ADC # [DU]

    # Propagate all noise sources to output
    Dn_shot_out = Vn_shot_pix * G_ADC # [DUrms]
    Dn_kTC_out = Vn_kTC_pix * G_ADC # [DUrms]
    Dn_Amp_out = Vn_Amp_Ampin * G_ADC # [DUrms]

    # Sum all noises at output
    Dntot_out = np.sqrt(Dn_shot_out**2 + Dn_kTC_out**2 + Dn_Amp_out**2 + Dn_quant_out**2) # [DUrms]

    # Compute SNR
    SNR = 20*np.log10(Dsig_out/Dntot_out) # [dB]
    
    
    ## Compute other results
    
    # Well capacity
    Nwell = Ci * Vmax / q # [#e-]
    
    # Read noise in electrons
    Nnread = Dntot_out[0] / G_ADC * Ci / q # [#e-]
    
    # Max SNR
    SNRmax = np.amax(SNR) # [dB]

    # Dynamic range
    DR = 20*np.log10(np.amax(Dsig_out)/Dntot_out[0]) # [dB]
    
    # Saturation time
    tSat = Nwell * q / Itot # [s]
    
    # Read time
    tRead = Npix * Res_ADC / Rdata # [s]
    
    # Max frame rate (no integration)
    FRmax0 = 1/tRead # [fps]
    
    # Max frame rate (full well, IWR)
    FRmax_MW_IWR = 1/np.maximum(tSat,tRead) # [fps]
    
    # Max frame rate (full well, ITR)
    FRmax_MW_ITR = 1/(tSat+tRead) # [fps]
    
    
    ## Define limiting noise string
    
    # Determine limiting noise
    DnmaxIndex = np.argmax((Dn_kTC_out[0],Dn_Amp_out[0],Dn_quant_out[0]))
    
    # Define string
    if DnmaxIndex == 0:
        LimitingNoise = 'reset'
    elif DnmaxIndex == 1:
        LimitingNoise = 'amplifier'
    elif DnmaxIndex == 2:
        LimitingNoise = 'quantization'    
    

    ## "Calculated Results" section heading and text results
    CalcResults = widgets.HTML(value = "<br><p style='font-size:16pt'><b>Calculated Results</b></p>"
        "<b>Well capacity</b>: " + str(round(Nwell/1e6,2)) + " Me-<br>"
        "<b>Read noise</b>: " + str(int(Nnread)) + " e-<br>"
        "&emsp;Limited by <b><i>" + LimitingNoise + "</i></b> noise<br>"
        "<b>Max SNR</b>: " + str(round(SNRmax,1)) + " dB<br>"
        "<b>Dynamic range</b>: " + str(round(DR,1)) + " dB<br>"
        "<b>Max frame rate (no integration)</b>: " + str(round(FRmax0,1)) + " fps<br>"
        "<b>Max frame rate (full well, IWR)</b>: " + str(round(FRmax_MW_IWR,1)) + " fps<br>"
        "<b>Max frame rate (full well, ITR)</b>: " + str(round(FRmax_MW_ITR,1)) + " fps<br>"
    )
    display(CalcResults)    
    
    ## Plot all noise sources (with sum) and signal, at output and referred to input
    plt.figure(1,figsize=(10,7));
    ax1 = plt.subplot()
    ax1.loglog(t,Dn_shot_out,'-g',label='Shot noise')
    ax1.loglog(t,Dn_kTC_out,'-m',label='Reset noise')
    ax1.loglog(t,Dn_Amp_out,'-r',label='Amplifier noise')
    ax1.loglog(t,Dn_quant_out,'-b',label='Quantization noise')
    ax1.loglog(t,Dntot_out,'-k',label='TOTAL noise')
    ax1.loglog(t,Dsig_out,'--k',label='Signal')
    ax1.grid()
    ax1.set_xlabel('Integration time [s]')
    ax1.set_ylabel('Output noise [DU]')
    ax1.legend(fontsize='medium')
    plt.title('Noise budget')

    
    # Integration time [s] to signal [#e-] conversion function for secondary x-axis
    def t2sig(x):
        return x * Isig / q
    
    # Signal [#e-] to integration time [s] conversion function for secondary x-axis
    def sig2t(x):
        return x * q / Isig
    
    # Output [DU] to input [#e-] conversion function for secondary y-axis
    def out2in(x):
        return x / G_ADC * Ci / q
    
    # Input [#e-] to output [DU] conversion function for secondary y-axis
    def in2out(x):
        return x * q / Ci * G_ADC
    
    # Secondary axes
    secaxx = ax1.secondary_xaxis('top', functions=(t2sig, sig2t)) # x-axis in units of #e-
    secaxy = ax1.secondary_yaxis('right', functions=(out2in, in2out)) # y-axis in units of #e-
    secaxx.set_xlabel('Signal [#e-]')
    secaxy.set_ylabel('Input-referred noise [#e-]')

    plt.figure(2,figsize=(10,7));
    plt.semilogx(t,SNR)
    plt.grid()
    plt.xlabel('Integration time [s]')
    plt.ylabel('SNR [dB]')
    plt.title('Signal-to-noise ratio')
    
    
# Description width style
style = {'description_width': 'initial'}

# Main interact() call
interact(plotfun,Isig = widgets.FloatText(value=1e-9, description=r'$I_{\rm sig}$ (A)'), # [A], Input signal current
    Idk = widgets.FloatText(value=1e-15, description=r'$I_{\rm dark}$ (A)'), # [A], Dark current
    Vn_Amp_Ampin = widgets.FloatText(value=100e-6, description=r'$V^n_{\rm amp}$ (V$_{\rm rms}$)'), # [Vrms], Amplifier noise referred to amplifier input
    Rdata = widgets.FloatText(value=5000, description='Data rate (Mbps)', style=style), # [Mbps], DROIC total data transmission rate
    Npix = widgets.FloatText(value=1, description=r'Array size (MPix)', style=style), # [Mpix], Total array size (number of pixels)
    T = widgets.FloatSlider(value=80, min=0, max=400, step=1, description=r'$T$ (K)'), # [K], Operating temperature
    CifF = widgets.FloatSlider(value=100, min=1, max=1000, step=1, description=r'$C_{\rm int}$ (fF)'), # [fF], Integration capacitance
    Vmax = widgets.FloatSlider(value=2, min=0.001, max=10, step=0.1, description=r'$V_{\rm max}$ (V)'), # [V], Maximum integration node voltage
    Res_ADC = widgets.IntSlider(value=14, min=1, max=30, description='ADC resolution (# bits)', style=style), # [# bits], ADC resolution   
);

HTML(value="<p style='font-size:16pt'><b>Set Parameters</b></p>")

interactive(children=(FloatText(value=1e-09, description='$I_{\\rm sig}$ (A)'), FloatText(value=1e-15, descrip…

## Code by: Zach Korth, Senseeker Engineering